# ⚡ Quick Dip ⚡

Here we are looking at the **inside** of the `multiclass F1 score` in PyTorch.  
Specifically, the **'macro'** score 🧠.

### 🎯 Macro F1:
> Calculate metrics for each class separately and return their unweighted mean.  
> Classes with 0 true and predicted instances are ignored 🚫.

I **want** the result to be given as a **log string** in Python,  
but the string is the **raw markdown code** 📜.

They claim that the function will **ignore non-present classes**,  
however, we are about to **check** here 🕵️‍♂️.

We are checking because **PyTorch** likes to throw these warnings ⚠️:
```WARNING:root:Warning: Some classes do not exist in the target. F1 scores for these classes will be cast to zeros.```


And if we cast the F1 score to zero and still counted the class,  
that would royally **mess up** the scores! 😱 So we will **double-check** here 🔍.

In [19]:
import torch
from torcheval.metrics.functional import multiclass_f1_score


In [13]:
target = torch.tensor([2,2,2,2,1,1,1,0,0])
pred = torch.tensor([2,2,2,1,1,1,2,0,0])

multiclass_f1_score(pred, target, num_classes=3, average=None)

tensor([1.0000, 0.6667, 0.7500])

In [18]:
target = torch.tensor([2,2,2,2,1,1,1,0,0])
pred = torch.tensor([2,2,2,1,1,1,2,0,0])

multiclass_f1_score(pred, target, num_classes=3, average="macro")

tensor(0.8056)

So if we remove all the 0 instances here, then the **macro F1** should be:

$$
\frac{(0.75 + 0.6667)}{2} = 0.70833
$$

and not:

$$
\frac{(0.75 + 0.6667 + 0)}{3} = 0.4722
$$

In [15]:
target = torch.tensor([2,2,2,2,1,1,1])
pred = torch.tensor([2,2,2,1,1,1,2])

multiclass_f1_score(pred, target, num_classes=3, average='macro')

tensor(0.7083)

So it works properly!! WHOOP WHOOP